In [1]:
import numpy as np
import pandas as pd
import matplotlib
import requests

In [108]:
data = pd.read_csv("desktop/clean_data.csv")

In [115]:
len(data[data['ebitda_margin']>1])

27

In [110]:
data.head(3)

,Unnamed: 0,symbol,enterpriseValue,sector,country,description,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin
0,0,0001.HK,21597.256799,Industrials,HK,"CK Hutchison Holdings Limited, an investment h...",0.051154,38402.729002,21451.105507,12705.530323,0.157184,0.330850
1,1,0002.HK,5136.485887,Utilities,HK,"CLP Holdings Limited, an investment holding co...",0.047146,10908.819796,3330.785385,2378.104992,0.023875,0.217998
2,2,0003.HK,4062.479373,Utilities,HK,"The Hong Kong and China Gas Company Limited, t...",0.091021,5217.793781,2417.570978,1608.588899,0.082469,0.308289


In [111]:
data.columns

Index(['Unnamed: 0', 'symbol', 'enterpriseValue', 'sector', 'country',
       'description', 'returnOnCapitalEmployed', 'revenue', 'grossProfit',
       'ebitda', 'growth_rate', 'ebitda_margin'],
      dtype='object')

In [112]:
len(data[data['grossProfit']<=0])

77

In [124]:
len(data[data['returnOnCapitalEmployed']<0])

845

In [105]:
len(data)

4720

In [98]:
data_scaled = data

In [103]:
from sklearn.preprocessing import RobustScaler

r_scaler = RobustScaler() # Instanciate Robust Scaler

# r_scaler.fit(data_scaled[['revenue']]) # Fit scaler to feature

data_scaled[['returnOnCapitalEmployed','revenue','grossProfit','ebitda', 'growth_rate', 'ebitda_margin']] = r_scaler.fit_transform(data_scaled[['returnOnCapitalEmployed','revenue','grossProfit','ebitda', 'growth_rate', 'ebitda_margin']]) #Scale

data_scaled.head()

,Unnamed: 0,symbol,enterpriseValue,sector,country,description,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin
0,0,0001.HK,21597.256799,Industrials,HK,"CK Hutchison Holdings Limited, an investment h...",-0.130746,12.224571,20.719361,23.266980,0.811304,1.044539
1,1,0002.HK,5136.485887,Utilities,HK,"CLP Holdings Limited, an investment holding co...",-0.167135,3.307408,3.026069,4.190919,-0.292405,0.384473
2,2,0003.HK,4062.479373,Utilities,HK,"The Hong Kong and China Gas Company Limited, t...",0.231236,1.461624,2.134375,2.769526,0.192715,0.912582
3,4,0006.HK,10.274256,Utilities,HK,"Power Assets Holdings Limited, an investment h...",0.134654,-0.174526,-0.057181,1.523345,-0.427494,30.662680
4,5,0007.HK,7.441258,Technology,HK,Hong Kong Finance Investment Holding Group Lim...,1.463516,-0.208266,-0.222606,0.032786,2.241234,9.857435


In [117]:
# need to remove EBITDA margin >1
# need to remove gross profit < 0 
# need to change the negative ROCE to the average of the sector
data.shape

(4720, 12)

In [128]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import FunctionTransformer

In [142]:
# rounder = Pipeline([('NoNegative',FunctionTransformer(lambda x: 0 if x<0 else x)),
#                     ('scaler', RobustScaler())])
    
num_transformer = Pipeline([('imputer', SimpleImputer()),
                            ('scaler', RobustScaler())])
# Preprocessor
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['revenue','returnOnCapitalEmployed','grossProfit','ebitda', 'growth_rate', 'ebitda_margin']),
    ('cat_transformer', OneHotEncoder(), ['sector', 'country'])])
    

# Combine preprocessor and linear model in pipeline
final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', Ridge())])
final_pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['revenue',
                                                   'returnOnCapitalEmployed',
                                                   'grossProfit', 'ebitda',
                                                   'growth_rate',
                                                   'ebitda_margin']),
                                                 ('cat_transformer',
                                                  OneHotEncoder(),
                                                  ['sector', 'country'])])),
                ('linear_regression', Ridge())])

In [143]:
 # visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', RobustScaler())]),
                                 ['revenue', 'returnOnCapitalEmployed',
                                  'grossProfit', 'ebitda', 'growth_rate',
                                  'ebitda_margin']),
                                ('cat_transformer', OneHotEncoder(),
                                 ['sector', 'country'])])

In [144]:
final_pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['revenue',
                                                   'returnOnCapitalEmployed',
                                                   'grossProfit', 'ebitda',
                                                   'growth_rate',
                                                   'ebitda_margin']),
                                                 ('cat_transformer',
                                                  OneHotEncoder(),
                                                  ['sector', 'country'])])),
                ('linear_regression', Ridge())])

In [145]:
data['returnOnCapitalEmployed']=data['returnOnCapitalEmployed'].apply(lambda x: 0 if x<0 else x)
X = data.drop(columns=['enterpriseValue','symbol','description','Unnamed: 0'])
y = data['enterpriseValue']
X.head()

,sector,country,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin
0,Industrials,HK,0.051154,38402.729002,21451.105507,12705.530323,0.157184,0.330850
1,Utilities,HK,0.047146,10908.819796,3330.785385,2378.104992,0.023875,0.217998
2,Utilities,HK,0.091021,5217.793781,2417.570978,1608.588899,0.082469,0.308289
3,Utilities,HK,0.080384,173.121215,173.121215,933.929875,0.007559,5.394659
4,Technology,HK,0.226738,69.095143,3.704255,126.968486,0.329895,1.837589


In [146]:
final_pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['revenue',
                                                   'returnOnCapitalEmployed',
                                                   'grossProfit', 'ebitda',
                                                   'growth_rate',
                                                   'ebitda_margin']),
                                                 ('cat_transformer',
                                                  OneHotEncoder(),
                                                  ['sector', 'country'])])),
                ('linear_regression', Ridge())])

In [147]:
final_pipe.fit(X,y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['revenue',
                                                   'returnOnCapitalEmployed',
                                                   'grossProfit', 'ebitda',
                                                   'growth_rate',
                                                   'ebitda_margin']),
                                                 ('cat_transformer',
                                                  OneHotEncoder(),
                                                  ['sector', 'country'])])),
                ('linear_regression', Ridge())])

In [ ]:
import